In [ ]:
import os
import tensorflow as tf
from keras.models import load_model
from flask import Flask, request
from werkzeug import secure_filename

UPLOAD_FOLDER = '/Users/PChomeIM/uploads'
ALLOWED_EXTENSIONS = ['png']
GRAYSCALE_MAX_NUM = 255

def allowed_file(filename):
    return '.' in filename \
        and filename.rsplit('.')[1] in ALLOWED_EXTENSIONS
    
def convert_image_to_array(imagepath):
    filename_queue = tf.train.string_input_producer([imagepath])
    reader = tf.WholeFileReader()
    key, value = reader.read(filename_queue)
    myimg = tf.image.decode_png(value, channels=1)
    crop_myimg = tf.image.resize_image_with_crop_or_pad(myimg, 140, 140)
    resize_myimg = tf.image.resize_images(crop_myimg, [28, 28])
    
    with tf.Session() as sess:
        init = tf.global_variables_initializer()
        sess.run(init)
    
        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(coord=coord)
    
        image = resize_myimg.eval().reshape(1, 28, 28, 1)
        normalize_image = abs(GRAYSCALE_MAX_NUM - image).astype('float32') / GRAYSCALE_MAX_NUM

        coord.request_stop()
        coord.join(threads)
    
    return normalize_image
    
app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

#載入預測模型
model = load_model("/Users/PChomeIM/pywork/SaveModel/MnistCNNModel.h5")

@app.route('/upload', methods=['GET', 'POST'])
def upload():
    if request.method == 'POST':
        file = request.files['file']
        if file and allowed_file(file.filename):
            filename = secure_filename(file.filename)
            filepath = os.path.join(app.config['UPLOAD_FOLDER'], filename)
            file.save(filepath)
            
            normalize_image = convert_image_to_array(filepath)
            
            prediction = model.predict_classes(normalize_image)

            return 'The number of image is ' + str(prediction[0])
        else:
            return 'The format of image is not supported.'
    return '''
    <!doctype html>
    <title>Upload new File</title>
    <h1>Upload new File</h1>
    <form action="" method=post enctype=multipart/form-data>
      <p><input type=file name=file>
         <input type=submit value=Upload>
    </form>
    '''
        
if __name__ == '__main__':
    app.run()

Using TensorFlow backend.
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [14/Sep/2017 15:37:48] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [14/Sep/2017 15:37:53] "GET /upload HTTP/1.1" 200 -


1/1 [==============================] - 0s

127.0.0.1 - - [14/Sep/2017 15:38:29] "POST /upload HTTP/1.1" 200 -
